## Простая проверка орфографии на основе формулы Байеса

Вольный перевод - http://norvig.com/spell-correct.html

In [1]:
import re
from collections import defaultdict
import pickle

Задача ставится так пользователь вводит слово, например "титрадь". Мы хотим предложить правильное написание. Очевидно, что исправление может быть не единственным - например, на запрос "тоу" можно предложить одинаковые корректировки "ток" и "том". Так как исправление не единственны, то нам следует ввести вероятностную модель, отражающую вероятность той или иной корректировки для данного запроса. Для простоты ограничимся русским алфавитом.

In [2]:
alphabet = u'абвгдеёжзиклмнопрстуфхцчшщъыьэюя'

Предположим, что "статистика" по коллекции документов известна - слова в документах подсчитаны, а любое неизвестное слово мы видим в первый раз.
Документы могут быть произвольной природы. Например, для поискового движка - это все страницы в сети интернет.

In [3]:
stored_dict = pickle.load( open( "dicts/dictionary.p", "rb" ) )

In [4]:
vocab = defaultdict(lambda: 1)
for key, value in stored_dict.iteritems():
    vocab[key] += value

Если коллекция достаточно большая, в том смысле что она может представлять пространство элементарных исходов, то частоту, нормированную на величину коллекции, можно рассматривать как "вероятность".

Введем некоторый формализм - будем обозначать введеное пользователем слово как $w$, корректировку как $c$.
Имея на руках несколько корректировок, мы можем описать вероятность каждой как $P(c|w)$. Среди всех возможных корректировок мы выберем ту, вероятность которой максимальна.

По теореме Байеса:
$$P(c|w) = \frac{P(w|c)P(c)}{P(w)}$$

Так как корректировки мы рассматриваем для фиксированного слова, то $P(w)$ одинакова для всех. Тогда нас будет интересовать следующая величина:

$$P(c_i|w) = P(w|c_i)P(c_i), c_i \in c$$

Эти величины имеют удобную интерпретацию:
- $P(c_i)$ - априорная (безусловная) вероятность слова $c_i$ ("языковая модель");
- $P(w|c_i)$ - вероятность того, что пользователь мог ошибиться и ввести $w$, когда на самом деле он имел в виду $c_i$ ("модель ошибки").

$P(c_i)$ можем получить непосредственным обращением к словарю

In [5]:
vocab[u'город']/float(len(vocab.keys()))

0.0006325433343195568

Перейдем к модели ошибок. Сделаем предположение о том, что $w$ тем менее вероятно, чем больше правок требуется для "превращения" $w$ в $c$. Какого рода правки рассматриваются? Перестановка букв, удаление буквы, добавление буквы из алфавита, замена буквы.

In [6]:
# функция, возвращающая все слова, которые находятся на расстоянии одной правки от исходного слова
def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return list(set(deletes + transposes + replaces + inserts))

In [7]:
for el in edits1(u'мыло'):
    print el

мыляо
мыхло
ёыло
мычло
мюло
мылом
мэло
ныло
мылео
мыго
мыфо
мьло
мысло
мшло
мрыло
чыло
мщло
хыло
мылж
мылз
мылд
мыле
мяло
мылг
мыла
мылб
мыло
мыьо
мылм
мылн
мылк
мылл
мыли
мыюло
пыло
мкыло
моыло
мыуло
мъло
мцыло
мхло
мпыло
мылоз
мылё
мыгло
мкло
мылио
иыло
мыщло
мрло
мло
мылч
мылф
мылх
мычо
мылу
мсло
мыво
ьыло
мыля
мыль
мылэ
мчло
мылы
мылш
мылщ
муло
мыъло
мхыло
мтыло
мылос
зыло
мвыло
мыяо
мыко
мылъ
мтло
мьыло
мылно
амыло
мылоо
выло
мылло
мвло
аыло
лмыло
мылор
мылфо
мыцо
уыло
ъмыло
мыело
ммло
мызло
моло
мынло
миыло
мнло
щыло
мылёо
мылоу
мыыло
мыео
мыюо
мытло
мыбло
мило
муыло
мылоц
мёыло
мыало
мылэо
мпло
мыжло
мылдо
млло
мывло
мыно
мфло
мыяло
мцло
мылог
мылто
лыло
змыло
мдло
мыпло
мжло
мысо
мыфло
мело
сыло
мыэло
мылв
юмыло
мылао
мбыло
мало
мыдо
эмыло
мбло
мыило
мшыло
омыло
мзло
пмыло
еыло
мгло
мыкло
юыло
цмыло
ыыло
ъыло
мылп
мымо
жыло
мылое
мэыло
мыёо
мяыло
тмыло
мылц
мышло
мыро
мылбо
мсыло
вмыло
мылоч
мылод
мылюо
кмыло
мжыло
было
мызо
мышо
мылмо
цыло
мыто
ьмыло
тыло
смыло
маыло
мылъо
щмы

In [8]:
# на расстоянии 2 правок
def edits2(word):
    return list(set(e2 for e1 in edits1(word) for e2 in edits1(e1)))

Число всех возможных правок может быть крайне велико. Для того, чтобы сократить число возможных проверок следует проверить их на наличие в словаре (напоминаем, что он представляет собой все возможное пространство исходов.)

In [9]:
def check_in_vocab(words):
    if isinstance(words, unicode):
        words = [words]
    return filter(lambda x: x in vocab,words)

Простейшая модель - считать, что правки длины один гораздо более вероятны, нежели правки длины два, правки длины два ... и так далее.

In [10]:
def correct(word):
    candidates = check_in_vocab([word]) or check_in_vocab(edits1(word)) or check_in_vocab(edits2(word)) or [word]
    return max(candidates, key=vocab.get)

Так как словарь маленький, то не все корректировки успешны. Однако чем более и более полным становится словарь, то тем выше становится качество модели.

In [11]:
print correct(u'лошать')

лошадь


In [12]:
print correct(u'силый')

милый


In [13]:
print correct(u'балото')

болото


In [14]:
print correct(u'тилифон')

телефон
